# Camada Bronze:

---

## Tabela Avaliações:

### Objetivo:

Entender a padronização e comportamento dos dados para criar a view avaliacoes_bronze a partir da camada RAW. 

### Fonte:

`workspace.default.avaliacoes`

### Destino:

`workspace.olist_bronze.avaliacoes_bronze`

In [0]:
-- Garantindo a existência do SCHEMA para armazenamento:

CREATE SCHEMA IF NOT EXISTS workspace.olist_bronze;


In [0]:
-- Verificando a existência da tabela:
SELECT * FROM workspace.default.avaliacoes;

In [0]:
-- Verificando quantas linhas tem na tabela:
SELECT COUNT(*) AS LINHAS
FROM workspace.default.avaliacoes;

In [0]:
-- Verificando os tipos de dados:
DESC workspace.default.avaliacoes;

Os tipos de dados estão corretamente definidos, com exceção da coluna `review_score`, que está tipificada como `BIGINT`. Considerando que a pontuação máxima é 5, esse tipo é desnecessariamente grande e resulta em uso ineficiente de memória. Dessa forma, a coluna pode ser convertida para `INT`, mantendo a semântica dos dados e reduzindo o consumo de memória.

In [0]:
-- Verificando missing values em cada coluna:
SELECT
      SUM(CASE WHEN review_id IS NULL THEN 1 ELSE 0 END) AS review_id_null,
      SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) AS order_id_null,
      SUM(CASE WHEN review_score IS NULL THEN 1 ELSE 0 END) AS review_score_null,
      SUM(CASE WHEN review_comment_title IS NULL THEN 1 ELSE 0 END) AS review_comment_title_null,
      SUM(CASE WHEN review_comment_message IS NULL THEN 1 ELSE 0 END) AS review_comment_message_null,
      SUM(CASE WHEN review_creation_date IS NULL THEN 1 ELSE 0 END) AS review_creation_date_null,
      SUM(CASE WHEN review_answer_timestamp IS NULL THEN 1 ELSE 0 END) AS review_answer_timestamp_null
FROM workspace.default.avaliacoes;

In [0]:
-- Verificando a proporção de nulos:
SELECT
      AVG(CASE WHEN review_id IS NULL THEN 100.0 ELSE 0 END) AS review_id_null_pct,
      AVG(CASE WHEN order_id IS NULL THEN 100.0 ELSE 0 END) AS order_id_null_pct,
      AVG(CASE WHEN review_score IS NULL THEN 100.0 ELSE 0 END) AS review_score_null_pct,
      AVG(CASE WHEN review_comment_title IS NULL THEN 100.0 ELSE 0 END) AS review_comment_title_null_pct,
      AVG(CASE WHEN review_comment_message IS NULL THEN 100.0 ELSE 0 END) AS review_comment_message_null_pct,
      AVG(CASE WHEN review_creation_date IS NULL THEN 100.0 ELSE 0 END) AS review_creation_date_null_pct,
      AVG(CASE WHEN review_answer_timestamp IS NULL THEN 100.0 ELSE 0 END) AS review_answer_timestamp_null_pct
FROM workspace.default.avaliacoes;


As colunas `review_id`, `order_id` e `review_score`, que são as mais relevantes para a criação de relacionamentos e para o cálculo de métricas, apresentam uma baixa porcentagem de valores nulos. 

Dessa forma, a exclusão desses registros não introduz viés significativo nos dados.

In [0]:
-- Verificando valores inválidos em cada coluna:
SELECT 
      SUM(CASE WHEN review_id IS NOT NULL AND NOT review_id RLIKE '^[A-Za-z0-9]+$' THEN 1 ELSE 0 END) AS review_id_invalid,

      SUM(CASE WHEN order_id IS NOT NULL AND NOT order_id RLIKE '^[A-Za-z0-9]+$' THEN 1 ELSE 0 END) AS order_id_invalid,

      SUM(CASE WHEN review_score IS NOT NULL AND review_score NOT BETWEEN 1 AND 5 THEN 1 ELSE 0 END) AS review_score_invalid,

      SUM(CASE WHEN review_comment_title IS NOT NULL AND NOT review_comment_title RLIKE '^[A-Za-z0-9À-ÿ '',.!?;:-]+$' THEN 1 ELSE 0 END) AS review_comment_title_invalid,

      SUM(CASE WHEN review_comment_message IS NOT NULL AND NOT review_comment_message  RLIKE '^[A-Za-z0-9À-ÿ '',.!?;:-]+$' THEN 1 ELSE 0 END) AS review_comment_message_invalid,

      SUM(CASE WHEN review_creation_date IS NOT NULL AND to_timestamp(review_creation_date, 'yyyy-MM-dd HH:mm:ss') IS NULL THEN 1 ELSE 0 END) AS review_creation_date_invalid,

      SUM(CASE WHEN review_answer_timestamp IS NOT NULL AND to_timestamp(review_answer_timestamp, 'yyyy-MM-dd HH:mm:ss') IS NULL THEN 1 ELSE 0 END) AS review_answer_timestamp_invalid

FROM workspace.default.avaliacoes;

Para a análise e a modelagem, a coluna de maior importância é `review_score`, que apresenta apenas um valor inválido, distinto de nulo. As colunas de identificação (IDs) possuem alguns valores inválidos, não somam uma quantidade alarmante, porém para criação de relacionamentos é interessante que a coluna `order_id` possua valores coerentes. 

Já os comentários e títulos com valores inválidos necessitam de uma investigação para entender o que classifica um comentário como inválido.

In [0]:
-- Verificando os valores inválidos das colunas de identificação (IDs)
SELECT 
      review_id,
      order_id
FROM workspace.default.avaliacoes
WHERE review_id NOT RLIKE '^[A-Za-z0-9]+$'
OR order_id NOT RLIKE '^[A-Za-z0-9]+$';

Conforme é possível observar na query acima, as colunas `review_id` e `order_id` contêm valores que deveriam estar preenchendo as colunas correspondentes à data, título do comentário e mensagem do comentário.

A realocação desses valores para as colunas corretas resultaria em registros com `order_id` nulo, tornando-os inutilizáveis para a criação de relacionamentos. Dessa forma, considerando que a quantidade de registros afetados não é significativa, a alternativa mais adequada é a exclusão desses registros.

In [0]:
-- Verificando os valores inválidos das colunas de comentários
SELECT 
      review_comment_title,
      review_comment_message
FROM workspace.default.avaliacoes
WHERE review_comment_title NOT RLIKE '^[A-Za-z0-9À-ÿ '',.!?;:-]+$'
OR review_comment_message NOT RLIKE '^[A-Za-z0-9À-ÿ '',.!?;:-]+$';

Observa-se que, na maioria dos casos, o desalinhamento dos valores está associado à presença de emojis, não demandando ações de tratamento.geral os valores parecem ter ficado fora por incluir o uso de emojis, não sendo necessário tratamento.

In [0]:
-- Criando a view avaliacoes_bronze:
CREATE OR REPLACE VIEW workspace.olist_bronze.avaliacoes_bronze AS
SELECT * 

FROM workspace.default.avaliacoes;